In [1]:
from pyfaidx import Fasta
import pandas as pd
from tqdm import tqdm

In [6]:
import os
print(os.listdir(r"C:\Users\saleh\561_project\data\chromFa"))

['chr1.fa', 'chr10.fa', 'chr11.fa', 'chr12.fa', 'chr13.fa', 'chr14.fa', 'chr15.fa', 'chr16.fa', 'chr17.fa', 'chr18.fa', 'chr19.fa', 'chr2.fa', 'chr20.fa', 'chr21.fa', 'chr22.fa', 'chr3.fa', 'chr4.fa', 'chr5.fa', 'chr6.fa', 'chr7.fa', 'chr8.fa', 'chr9.fa', 'chrX.fa', 'chrY.fa']


In [11]:
with open(r"C:/Users/saleh/561_project/data/chromFa/chr1.fa") as f:
    print(f.readline())

>chr1



In [12]:
import os
os.path.isdir(r"C:/Users/saleh/561_project/data/chromFa/")


True

In [13]:
from pyfaidx import Fasta
test_chr1 = Fasta(r"C:/Users/saleh/561_project/data/chromFa/chr1.fa")
print(list(test_chr1.keys())[:5])


['chr1']


MERGING ALL FASTA INTO ONE all chromosomes, into one large fasta file. 

In [14]:
import os

src = r"C:/Users/saleh/561_project/data/chromFa"
out = r"C:/Users/saleh/561_project/data/hg19.fa"

with open(out, "w") as outfile:
    for fname in sorted(os.listdir(src)):
        if fname.endswith(".fa"):
            with open(os.path.join(src, fname)) as infile:
                outfile.write(infile.read())

print("hg19.fa created!")


hg19.fa created!


In [15]:
from pyfaidx import Fasta

genome = Fasta(r"C:/Users/saleh/561_project/data/hg19.fa")

print(genome.keys())


odict_keys(['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21', 'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrX', 'chrY'])


In [16]:
from pyfaidx import Fasta
import pandas as pd
from tqdm import tqdm

# Load merged genome FASTA
genome = Fasta(r"C:/Users/saleh/561_project/data/hg19.fa")

# Load BED files
pos_bed = pd.read_csv(
    r"C:/Users/saleh/561_project/data/CTCF_positive.bed",
    sep="\t",
    header=None,
    names=["chrom", "start", "end", "tf", "score", "strand"]
)

neg_bed = pd.read_csv(
    r"C:/Users/saleh/561_project/data/CTCF_negative.bed",
    sep="\t",
    header=None,
    names=["chrom", "start", "end", "tf", "score", "strand"]
)

def extract_sequences(df, genome, output_fasta):
    print(f"Extracting sequences → {output_fasta}")
    with open(output_fasta, "w") as out:
        for _, row in tqdm(df.iterrows(), total=len(df)):
            chrom = row["chrom"]
            start = int(row["start"])
            end   = int(row["end"])
            strand = row["strand"]

            # skip chromosomes not in FASTA
            if chrom not in genome.keys():
                continue

            seq = genome[chrom][start:end].seq.upper()

            if strand == "-":
                seq = seq.translate(str.maketrans("ACGT", "TGCA"))[::-1]

            out.write(f">{chrom}:{start}-{end}_{strand}\n{seq}\n")


# Extract positives
extract_sequences(
    pos_bed, genome,
    r"C:/Users/saleh/561_project/data/CTCF_positive.fasta"
)

# Extract negatives
extract_sequences(
    neg_bed, genome,
    r"C:/Users/saleh/561_project/data/CTCF_negative.fasta"
)

print("DONE — sequences extracted.")


Extracting sequences → C:/Users/saleh/561_project/data/CTCF_positive.fasta


100%|██████████| 95135/95135 [00:04<00:00, 20912.21it/s]


Extracting sequences → C:/Users/saleh/561_project/data/CTCF_negative.fasta


100%|██████████| 95135/95135 [00:05<00:00, 17684.23it/s]

DONE — sequences extracted.


gotta merge both positive and negative fasta and then in Rstudio we will use DNAshape package
then: that will give us the physical properties Helt, roll, etc.

BUT we want it all in one csv, that we can later use for training.




gpt: combing positive and negative fasta code

In [17]:
pos_fa = r"C:/Users/saleh/561_project/data/CTCF_positive.fasta"
neg_fa = r"C:/Users/saleh/561_project/data/CTCF_negative.fasta"

combined_fa = r"C:/Users/saleh/561_project/data/CTCF_all.fa"
label_csv    = r"C:/Users/saleh/561_project/data/CTCF_labels.csv"

# Create combined FASTA + label file
with open(combined_fa, "w") as out_f, open(label_csv, "w") as lab_f:

    lab_f.write("id,label\n")   # CSV header

    # --- POSITIVE SEQUENCES ---
    with open(pos_fa) as f:
        for line in f:
            if line.startswith(">"):
                seq_id = "pos_" + line[1:].strip()   # prefix with pos_
                out_f.write(">" + seq_id + "\n")
                lab_f.write(f"{seq_id},1\n")
            else:
                out_f.write(line)

    # --- NEGATIVE SEQUENCES ---
    with open(neg_fa) as f:
        for line in f:
            if line.startswith(">"):
                seq_id = "neg_" + line[1:].strip()   # prefix with neg_
                out_f.write(">" + seq_id + "\n")
                lab_f.write(f"{seq_id},0\n")
            else:
                out_f.write(line)

print("DONE — merged into CTCF_all.fa and created CTCF_labels.csv")


DONE — merged into CTCF_all.fa and created CTCF_labels.csv


In [18]:
pos_fa = r"C:/Users/saleh/561_project/data/CTCF_positive.fasta"
neg_fa = r"C:/Users/saleh/561_project/data/CTCF_negative.fasta"

combined_fa = r"C:/Users/saleh/561_project/data/CTCF_all.fa"
label_csv    = r"C:/Users/saleh/561_project/data/CTCF_labels.csv"

out_f = open(combined_fa, "w")
lab_f = open(label_csv, "w")
lab_f.write("id,label\n")

counter = 0

# --- POSITIVES ---
with open(pos_fa) as f:
    for line in f:
        if line.startswith(">"):
            counter += 1
            clean_id = line[1:].strip().replace(":", "_")
            seq_id = f"pos_{clean_id}_{counter}"
            out_f.write(">" + seq_id + "\n")
            lab_f.write(f"{seq_id},1\n")
        else:
            out_f.write(line)

# --- NEGATIVES ---
with open(neg_fa) as f:
    for line in f:
        if line.startswith(">"):
            counter += 1
            clean_id = line[1:].strip().replace(":", "_")
            seq_id = f"neg_{clean_id}_{counter}"
            out_f.write(">" + seq_id + "\n")
            lab_f.write(f"{seq_id},0\n")
        else:
            out_f.write(line)

out_f.close()
lab_f.close()

print(f"DONE — total sequences written: {counter}")


DONE — total sequences written: 190270
